In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import softmax

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
# Import train_tweets.csv and test_tweets.csv file
train_tweets = pd.read_csv('train_E6oV3lV.csv')
test_tweets = pd.read_csv('test_tweets_anuFYb8.csv')

In [4]:
train_tweets.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
train_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [6]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Data preprocessing
train_tweets.drop(columns = ['id'], inplace=True)
#train_tweets.drop('id')  # Remove unnecessary columns

# Load the dataset
#df = pd.read_csv("sentiment_dataset.csv")

# Define a function for text preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    # Perform stemming (optional) or lemmatization
    # Stemming reduces words to their root form (e.g., "running" becomes "run").
    # Lemmatization reduces words to their base or dictionary form (e.g., "better" becomes "good").
    # Choose one or neither based on your needs.
    stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]
    
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Rejoin the words into a cleaned text
    cleaned_text = " ".join(words)
    
    return cleaned_text

# Apply the text preprocessing function to the 'tweets' column
train_tweets['tweet'] = train_tweets['tweet'].apply(preprocess_text)

# Display the cleaned dataset
print(train_tweets.head())


   label                                              tweet
0      0  user father dysfunctional selfish drag kid dys...
1      0  user user thanks lyft credit use cause offer w...
2      0                                     bihday majesty
3      0                        model love u take u time ur
4      0                      factsguide society motivation


In [7]:

# Data splitting
#train, test = train_test_split(df, test_size=0.2, random_state=42)
train_tweets, val_tweets = train_test_split(train_tweets, test_size=0.2, random_state=42)

# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(list(train_tweets['tweet']), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(list(val_tweets['tweet']), truncation=True, padding=True, return_tensors='pt')

# Load pretrained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Fine-tuning (training) and evaluation steps go here

# Inference on new data
# For example, you can tokenize and classify new text using the trained model

# Reporting and visualization
# Analyze and visualize model performance


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
input_ids = train_encodings['input_ids']
labels = train_tweets['label']

In [9]:
print(input_ids)

tensor([[  101,  5310,  5310,  ...,     0,     0,     0],
        [  101,  5310, 21298,  ...,     0,     0,     0],
        [  101,  5958,  1043,  ...,     0,     0,     0],
        ...,
        [  101,  2304,  2934,  ...,     0,     0,     0],
        [  101, 12010,  4160,  ...,     0,     0,     0],
        [  101,  2191,  2111,  ...,     0,     0,     0]])


In [10]:
print(labels)

12110    1
14081    0
1829     0
2769     0
31818    0
        ..
29802    0
5390     0
860      1
15795    0
23654    0
Name: label, Length: 25569, dtype: int64


In [11]:
# Convert input_ids and labels to PyTorch tensors
input_ids = torch.tensor(input_ids)
#labels = torch.tensor(label)

C:\Users\eklou\AppData\Local\Temp\ipykernel_12508\167678699.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)


In [12]:


# Create a TensorDataset
dataset = TensorDataset(input_ids)

# Create a DataLoader
batch_size = 32  # Adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size)

In [13]:
# Assuming you have a DataLoader named 'dataloader' containing test data

# Set the model to evaluation mode
model.eval()

# Define lists to store predictions
all_predictions = []

# Iterate over batches in the data loader
for batch in dataloader:
    input_ids, labels = batch  # Extract input IDs and labels from the batch

    # Move data to the device (e.g., GPU if available)
    input_ids = input_ids.to(device)

    # Perform forward pass (classification) using the model
    with torch.no_grad():
        outputs = model(input_ids)

    # Process the outputs to obtain predictions (example assumes binary classification)
    predicted_probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Assuming binary classification
    predicted_labels = (predicted_probabilities > 0.5).long()  # Convert probabilities to labels

    # Append predictions to the list
    all_predictions.extend(predicted_labels.cpu().numpy())  # Assuming CPU is used

# Output the predictions
print("Predictions:", all_predictions)


ValueError: not enough values to unpack (expected 2, got 1)